# Фаза 2 • Неделя 11 • Понедельник
## Рекомендательные системы
### Классические подходы 

### Задание

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.express as px

### Загрузка данных

Запусти ячейку ниже, чтобы загрузить данные. 

In [2]:
CURR_PATH = os.path.dirname("__name__")

In [3]:
# 1. Users dataset
u_cols = ["user_id", "age", "sex", "occupation", "zip_code"]
users = pd.read_csv(
    os.path.join(CURR_PATH, "data", "ml-100k", "u.user"),
    sep="|",
    names=u_cols,
    encoding="latin-1",
    parse_dates=True,
    header=None,
)
# 2. Rating dataset
r_cols = ["user_id", "movie_id", "rating", "unix_timestamp"]
ratings = pd.read_csv(
    os.path.join(CURR_PATH, "data", "ml-100k", "u.data"),
    sep="\t",
    names=r_cols,
    encoding="latin-1",
)

# 3.Movies Dataset
m_cols = [
    "movie_id",
    "title",
    "release_date",
    "video_release_date",
    "imdb_url",
    "unknown",
    "Action",
    "Adventure",
    "Animation",
    "Children's",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]
movies = pd.read_csv(
    os.path.join(CURR_PATH, "data", "ml-100k", "u.item"),
    sep="|",
    names=m_cols,
    encoding="latin-1",
).drop(["video_release_date", "unknown"], axis=1)

ratings = ratings.merge(movies[["movie_id", "title"]], how="left", on="movie_id")

In [4]:
users.shape, ratings.shape, movies.shape

((943, 5), (100000, 5), (1682, 22))

In [5]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [7]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)


## 📊 Exploratory data analysis / Разведывательный анализ данных

Чтобы что-то рекомендовать, всегда стоит узнать о пользователях побольше. В этом разделе тебе необходимо визуализировать распределения разных признаков твоей рекомендательной системы. 

In [102]:
def configure_plotly_theme(fig):
    """Функция для применения единого стиля ко всем графикам"""
    fig.update_layout(
        plot_bgcolor="white",
        paper_bgcolor="white",
        font=dict(size=14),
        xaxis=dict(
            title_font=dict(size=20),
            tickfont=dict(size=16),
            gridcolor="lightgray",
            gridwidth=1,
            griddash="dash",
            showline=True,
            linecolor="black",
            linewidth=1,
        ),
        yaxis=dict(
            title_font=dict(size=20),
            tickfont=dict(size=16),
            gridcolor="lightgray",
            gridwidth=1,
            griddash="dash",
            showline=True,
            linecolor="black",
            linewidth=1,
        ),
        title_font=dict(size=24, family="Arial", weight="bold"),
    )
    fig.update_xaxes(
        # title_text="Название оси X",
        title_font=dict(size=20, family="Arial")
    )
    fig.update_yaxes(
        # title_text="Название оси Y",
        title_font=dict(size=20, family="Arial")
    )

    return fig

### Пользователь 👨

Визуализируй следующие распределения: 
- пола (`barplot`)

In [ ]:
sex_distrib = users.groupby("sex").count()["user_id"]
fig_1 = px.bar(x=sex_distrib.index, y=sex_distrib, title="Распределение пола")
fig_1.update_layout(
    width=800,  # ширина в пикселях
    height=600,  # высота в пикселях
    # autosize=False,  # отключаем авторазмер
    xaxis_title="Пол",
    yaxis_title="Количество пользователей",
    # title_font=dict(size=24, family="Arial", weight="bold"),
)
fig_1 = configure_plotly_theme(fig_1)
fig_1.update_traces(marker=dict(line=dict(width=2, color="black")))
fig_1.update_traces(
    texttemplate="%{y}",  # Формат текста (значение y)
    textposition="outside",  # Положение: 'outside', 'inside', 'auto', 'none'
    textfont=dict(
        size=18, family="Arial", color="black"
    ),  # Размер шрифта  # Цвет текста
)
fig_1.update_layout(
    xaxis=dict(
        showticklabels=True,
        # tickmode="linear",
        # ticks="outside",  # Ticks снаружи: 'outside', 'inside', ''
        # ticklen=6,  # Длина ticks
        # tickwidth=2,  # Толщина ticks
        # tickcolor="black",  # Цвет ticks
        # dtick=100,
        # tickmode='array',  # Режим ручной установки
        # tickvals=[0, 1, 2, 3],  # Позиции ticks
        # ticktext=['Категория A', 'Категория B', 'Категория C', 'Категория D']  # Подписи
    ),
    yaxis=dict(
        showticklabels=True,
        tickmode="linear",
        dtick=100,
        ticks="inside",  # Ticks снаружи: 'outside', 'inside', ''
        ticklen=6,  # Длина ticks
        tickwidth=2,  # Толщина ticks
        # tickmode="linear",
        # dtick=5
    ),
)
fig_1.show()

- возраста (`hist`)

In [121]:
fig_2 = px.histogram(users["age"].astype(int), nbins=70, title="Распределение возраста")
fig_2.update_layout(
    width=800,  # ширина в пикселях
    height=600,  # высота в пикселях
    # autosize=False,  # отключаем авторазмер
    xaxis_title="Возраст",
    yaxis_title="Количество пользователей",
    # title_font=dict(size=24, family="Arial", weight="bold"),
    showlegend=False,
)
fig_2.update_layout(
    xaxis=dict(
        showticklabels=True,
        tickmode="linear",
        ticks="outside",  # Ticks снаружи: 'outside', 'inside', ''
        ticklen=6,  # Длина ticks
        tickwidth=2,  # Толщина ticks
        tickcolor="black",  # Цвет ticks
        dtick=10,
        # tickmode='array',  # Режим ручной установки
        # tickvals=[0, 1, 2, 3],  # Позиции ticks
        # ticktext=['Категория A', 'Категория B', 'Категория C', 'Категория D']  # Подписи
    ),
    yaxis=dict(
        showticklabels=True,
        tickmode="linear",
        dtick=5,
        ticks="inside",  # Ticks снаружи: 'outside', 'inside', ''
        ticklen=6,  # Длина ticks
        tickwidth=2,  # Толщина ticks
        # tickmode="linear",
        # dtick=5
    ),
)
fig_2 = configure_plotly_theme(fig_2)
fig_2.update_traces(marker=dict(line=dict(width=1, color="black")))
fig_2.show()

- профессии (`barplot`)

In [123]:
occupations_distrib = users.groupby("occupation").count()["user_id"]
occupations_distrib = occupations_distrib.sort_values(ascending=False)
fig_3 = px.bar(
    x=occupations_distrib.index, y=occupations_distrib, title="Распределение профессий"
)
fig_3.update_layout(
    width=800,  # ширина в пикселях
    height=600,  # высота в пикселях
    # autosize=False,  # отключаем авторазмер
    xaxis_title="Профессия",
    yaxis_title="Количество пользователей",
    # title_font=dict(size=24, family="Arial", weight="bold"),
)
fig_3.update_traces(
    marker=dict(line=dict(width=1.5, color="black"))  # Толщина рамки  # Цвет рамки
)
fig_3.update_layout(
    xaxis=dict(tickangle=-45)  # Поворот на 45 градусов против часовой стрелки
)
fig_3.update_traces(
    texttemplate="%{y}",  # Формат текста (значение y)
    textposition="outside",  # Положение: 'outside', 'inside', 'auto', 'none'
    textfont=dict(
        size=14, family="Arial", color="black"
    ),  # Размер шрифта  # Цвет текста
)
fig_3.update_layout(
    xaxis=dict(
        showticklabels=True,
        # tickmode="linear",
        # ticks="outside",  # Ticks снаружи: 'outside', 'inside', ''
        # ticklen=6,  # Длина ticks
        # tickwidth=2,  # Толщина ticks
        # tickcolor="black",  # Цвет ticks
        # dtick=10,
        # tickmode='array',  # Режим ручной установки
        # tickvals=[0, 1, 2, 3],  # Позиции ticks
        # ticktext=['Категория A', 'Категория B', 'Категория C', 'Категория D']  # Подписи
    ),
    yaxis=dict(
        showticklabels=True,
        tickmode="linear",
        dtick=50,
        ticks="outside",  # Ticks снаружи: 'outside', 'inside', ''
        ticklen=6,  # Длина ticks
        tickwidth=2,  # Толщина ticks
        # tickmode="linear",
        # dtick=5
    ),
)
fig_3 = configure_plotly_theme(fig_3)
fig_3.show()

### Фильмы 🎥

Визуализируй следующие распределения: 

- количество фильмов каждого жанра (у нас есть много жанров фильмов: сколько фильмов у каждого жанра?)

In [ ]:
movies.head()

,movie_id,title,release_date,imdb_url,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [59]:
genres = movies.iloc[:, 4:]
print(genres.shape)
genres_film_count = genres.copy().apply(sum)
genres_film_count = genres_film_count.sort_values(ascending=False)

(1682, 18)


In [126]:
fig_4 = px.bar(
    x=genres_film_count.index,
    y=genres_film_count,
    title="Распределение фильмов по жанрам",
)
fig_4.update_layout(
    width=1000,  # ширина в пикселях
    height=600,  # высота в пикселях
    # autosize=False,  # отключаем авторазмер
    xaxis_title="Жанры",
    yaxis_title="Количество фильмов",
    # title_font=dict(size=24, family="Arial", weight="bold"),
)
fig_4.update_traces(
    marker=dict(line=dict(width=1.5, color="black"))  # Толщина рамки  # Цвет рамки
)
fig_4.update_layout(
    xaxis=dict(tickangle=-45)  # Поворот на 45 градусов против часовой стрелки
)
fig_4.update_traces(
    texttemplate="%{y}",  # Формат текста (значение y)
    textposition="outside",  # Положение: 'outside', 'inside', 'auto', 'none'
    textfont=dict(
        size=14, family="Arial", color="black"
    ),  # Размер шрифта  # Цвет текста
)
fig_4.update_layout(
    xaxis=dict(
        showticklabels=True,
        tickmode="linear",
        ticks="outside",  # Ticks снаружи: 'outside', 'inside', ''
        ticklen=6,  # Длина ticks
        tickwidth=2,  # Толщина ticks
        tickcolor="black",  # Цвет ticks
        dtick=1,
        # tickmode='array',  # Режим ручной установки
        # tickvals=[0, 1, 2, 3],  # Позиции ticks
        # ticktext=['Категория A', 'Категория B', 'Категория C', 'Категория D']  # Подписи
    ),
    yaxis=dict(
        showticklabels=True,
        tickmode="linear",
        dtick=100,
        ticks="outside",  # Ticks снаружи: 'outside', 'inside', ''
        ticklen=6,  # Длина ticks
        tickwidth=2,  # Толщина ticks
        # tickmode="linear",
        # dtick=5
    ),
)
fig_4 = configure_plotly_theme(fig_4)
fig_4.show()

- распределение количества жанров у разных фильмов (какие-то фильмы принадлежат одному жанру, какие-то сразу нескольким)

In [89]:
genres_distrib = genres.copy()
genres_distrib["id"] = movies["movie_id"]
genres_distrib.set_index("id", inplace=True)
genres_distrib = genres_distrib.T
# genres_distrib.reset_index(inplace=True)
genres_distrib.head()
genres_sum = genres_distrib.apply(sum)
genres_sum = genres_sum.reset_index()
genres_sum.head()

,id,0
0,1,3
1,2,3
2,3,1
3,4,3
4,5,3


In [91]:
for_barplot = genres_sum.groupby(0).count()
for_barplot

,id
0,
0,2
1,831
2,569
3,215
4,51
5,11
6,3


In [128]:
# genres_sum.head()
fig_5 = px.bar(
    x=for_barplot.index,
    y=for_barplot["id"],
    title="Распределение количества жанров",
)
fig_5.update_layout(
    width=1000,  # ширина в пикселях
    height=600,  # высота в пикселях
    # autosize=False,  # отключаем авторазмер
    xaxis_title="Количество жанров",
    yaxis_title="Количество фильмов",
    # title_font=dict(size=24, family="Arial", weight="bold"),
)
fig_5.update_traces(
    marker=dict(line=dict(width=1.5, color="black"))  # Толщина рамки  # Цвет рамки
)
# fig_5.update_layout(
#     xaxis=dict(tickangle=-45)  # Поворот на 45 градусов против часовой стрелки
# )
fig_5.update_traces(
    texttemplate="%{y}",  # Формат текста (значение y)
    textposition="outside",  # Положение: 'outside', 'inside', 'auto', 'none'
    textfont=dict(
        size=14, family="Arial", color="black"
    ),  # Размер шрифта  # Цвет текста
)
fig_5.update_layout(
    xaxis=dict(
        showticklabels=True,
        tickmode="linear",
        ticks="outside",  # Ticks снаружи: 'outside', 'inside', ''
        ticklen=6,  # Длина ticks
        tickwidth=2,  # Толщина ticks
        tickcolor="black",  # Цвет ticks
        dtick=1,
        # tickmode='array',  # Режим ручной установки
        # tickvals=[0, 1, 2, 3],  # Позиции ticks
        # ticktext=['Категория A', 'Категория B', 'Категория C', 'Категория D']  # Подписи
    ),
    yaxis=dict(
        showticklabels=True,
        tickmode="linear",
        dtick=100,
        ticks="outside",  # Ticks снаружи: 'outside', 'inside', ''
        ticklen=6,  # Длина ticks
        tickwidth=2,  # Толщина ticks
        # tickmode="linear",
        # dtick=5
    ),
)
fig_5 = configure_plotly_theme(fig_5)
fig_5.show()

- корреляцонная матрица жанров (какие-то жанры близки между собой, а какие-то противоречивы, корреляционная матрица поможет понять, какие жанры связаны между собой больше всего)

In [129]:
genres.head()

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [142]:
corr_matrix = genres.corr()

fig_6 = px.imshow(
    corr_matrix,
    text_auto=".2f",  # Формат чисел с 2 знаками после запятой
    color_continuous_scale=["blue", "white", "red"],
    zmin=-1,  # Минимальное значение шкалы
    zmax=1,  # Максимальное значение шкалы
    title="Корреляционная матрица",
)

# Настройка внешнего вида
fig_6.update_layout(
    width=1000,
    height=1000,
    # xaxis_title="Переменные", yaxis_title="Переменные"
)
fig_6.update_layout(
    plot_bgcolor="white",
    paper_bgcolor="white",
    font=dict(size=16),
    xaxis=dict(
        title_font=dict(size=20),
        tickfont=dict(size=16),
        # gridcolor="lightgray",
        # gridwidth=1,
        # griddash="dash",
        # showline=True,
        # linecolor="black",
        # linewidth=1,
    ),
    yaxis=dict(
        title_font=dict(size=20),
        tickfont=dict(size=16),
        # gridcolor="lightgray",
        # gridwidth=1,
        # griddash="dash",
        # showline=True,
        # linecolor="black",
        # linewidth=1,
    ),
    title_font=dict(size=24, family="Arial", weight="bold"),
)
fig_6.update_xaxes(
    # title_text="Название оси X",
    title_font=dict(size=20, family="Arial")
)
fig_6.update_yaxes(
    # title_text="Название оси Y",
    title_font=dict(size=20, family="Arial")
)
fig_6.show()

# ❓

Найди два жанра, которые больше всего близки друг к другу и запиши ответ в ячейку ниже

> Children и Animation

<img src="https://icons.iconarchive.com/icons/icons8/windows-8/256/Programming-Github-icon.png" width=32 /> Пора сохранить изменения для __github__. После пуша распечатай результат команды `!git status` в ячейке ниже.

In [33]:
# code

### Анализ рейтинга 🔢
Рейтинг - самое информативное, что у тебя есть. 

1. Построй распределение количества оценок, поставленных пользователем. Это должна быть функция, которая принимает на вход `user_id`, а на выходе возвращает словарь вида: 
    
`{1: <число оценок 1, 2: <число оценок 2>, ..., 5 <число оценок 5>}`

На основе этой результата данной функции должен строиться словарь. 

In [23]:
def get_user_ratings(user_id: int) -> dict:
    # code
    return user_ratings

In [25]:
# code

2. Построй распределение средней оценки по пользователям. Для этого нужно узнать, какая средняя оценка у каждого пользователя, а после этого построить распределение. 

In [27]:
# code

3. Построй распределение количества оценок, поставленных фильму. Каждый фильм оценен разное количество раз. Узнай, сколько оценок у каждого фильма, а после визуализируй это на гистограмме.  

In [29]:
# code

4. Построй распределение средней оценки по фильмам. 

In [30]:
# code

<img src="https://icons.iconarchive.com/icons/icons8/windows-8/256/Programming-Github-icon.png" width=32 /> Пора сохранить изменения для __github__. После пуша распечатай результат команды `!git status` в ячейке ниже.

In [34]:
# code

### Рекомендации по популярности 🔝

Мы хотим рекомендовать фильмы с самой большой оценкой, которые при этом смотрели достаточно часто. Для этого нужно оценить величину, которую мы назовем __score__, значение которой будет вычисляться как логарифм числа оценок фильма, умноженный на среднюю оценку: 
$$score_{film} = \log n * \bar{r}_{film},$$
где $n$ - число оценок для фильма, $\bar{r}_{film}$ - средний рейтинг фильма. 

# ❓
Зачем мы берем логарифм от числа оценок фильма?

> ответ тут

Добавь колонки `num_ratings, mean_rating, score` в датафрейм `ratings` и найди топ-10 фильмов, которые всем точно стоит посмотреть.

In [ ]:
# code

## Контентный подход

### Похожесть фильмов между собой 🎥 - 🎥

Посчитай меры похожести каждой пары фильмов по жанрам. Будем рекомендовать фильмы, похожие на какой-нибудь из фильмов, который понравился пользователю. В результате должна получиться функция `recommend(user_id, top=10)`, которая должна выдавать `list` или `pd.DataFrame` из `<top>` фильмов, которые мы будем рекомендовать пользователю.

<details>
<summary>Что такое похожесть в текущем контексте?</summary>
Каждый фильм может быть представлен вектором, описывающим его принадлежность к жанрам. Близость этих векторов можно использовать как меру сходства двух фильмов. 
</details>


In [ ]:
def recommend(user_id, top=10):
    pass
    return top_films

### Похожесть пользователей между собой 🥸 - 🥴

Найди 10 самых похожих пользователей и рекомендуй текущему пользователю то, что понравилось наиболее близким к нему пользователям. В результате должна получиться функция `recommend_by_user(user_id, top=10)`, которая должна выдавать `list` или `pd.DataFrame` из `<top>` фильмов, которые мы будем рекомендовать пользователю. 

In [ ]:
def recommend_by_user(user_id, top=10):
    pass
    return top_films

### scikit-surprice


Создай `user-item` матрицу (по строкам – пользователи, по столбцам – фильмы, на пересечении – оценки). Примени `SVD` для решения задачи прогнозирования оценки. 

> https://surpriselib.com/


Реализуй предсказание рейтинга пользователя с помощью алгоритма [Nonnegative Matrix Factorization](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.NMF). Для этого понадобится библиотека [scikit-surpice](https://surprise.readthedocs.io/en/stable/index.html)

In [ ]:
# code

### Slope One

Реализуй подход Slope One (пример есть в слайдах лекции или в [статье](https://www.researchgate.net/publication/1960789_Slope_One_Predictors_for_Online_Rating-Based_Collaborative_Filtering))

In [ ]:
# code

<img src="https://icons.iconarchive.com/icons/icons8/windows-8/256/Programming-Github-icon.png" width=32 /> Сделай `commit + push` на  __github__. 

In [ ]:
# code